<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-both-models-in-tandem" data-toc-modified-id="Using-both-models-in-tandem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using both models in tandem</a></span><ul class="toc-item"><li><span><a href="#Facial-Keypoints-model" data-toc-modified-id="Facial-Keypoints-model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Facial Keypoints model</a></span></li><li><span><a href="#Emotion-Detection-Model" data-toc-modified-id="Emotion-Detection-Model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Emotion Detection Model</a></span></li></ul></li><li><span><a href="#Loading-test-split(of-emotion-data)" data-toc-modified-id="Loading-test-split(of-emotion-data)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Loading test split(of emotion data)</a></span></li><li><span><a href="#Deploying-the-models-using-Tensorflow-2.0-serving" data-toc-modified-id="Deploying-the-models-using-Tensorflow-2.0-serving-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Deploying the models using Tensorflow 2.0 serving</a></span></li></ul></div>

In [18]:
import pandas as pd
import numpy as np
import json
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import matplotlib
%matplotlib inline

# Using both models in tandem

Now that we have trained and tuned Deep Learning models for both the facial keypoints and emotion detection tasks, we proceed to combining the models and making multiple predictions for a given image set

## Facial Keypoints model

In [2]:
with open('trained_models/facial_keypoints_resnet_model.json', 'r') as json_file:
    json_savedModel= json_file.read()
    
# load the model architecture 
facial_keypoints_resnet_model = tf.keras.models.model_from_json(json_savedModel)
facial_keypoints_resnet_model.load_weights('trained_models/facial_keypoints_resnet_model_weights.hdf5')
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
facial_keypoints_resnet_model.compile(loss="mean_squared_error", optimizer= adam , metrics = ['accuracy'])

## Emotion Detection Model

In [3]:
with open('emotion.json', 'r') as json_file:
    json_savedModel= json_file.read()
    
# load the model architecture 
emotion_model = tf.keras.models.model_from_json(json_savedModel)
emotion_model.load_weights('weights_emotions.hdf5')
emotion_model.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

Let us define a function which uses both the models and makes predicitions on a given data

In [4]:
# Gettting columns of facial keypoints dataframe to apply to results dataframe

with open('trained_models/keypoints_df_columns.json') as col_file:
    columns = json.load(col_file)
    
print(columns)

['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y', 'left_eye_inner_corner_x', 'left_eye_inner_corner_y', 'left_eye_outer_corner_x', 'left_eye_outer_corner_y', 'right_eye_inner_corner_x', 'right_eye_inner_corner_y', 'right_eye_outer_corner_x', 'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x', 'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x', 'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x', 'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x', 'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y', 'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x', 'mouth_right_corner_y', 'mouth_center_top_lip_x', 'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y']


In [5]:
def predict(test_imgs):
    
    # Make predictions using facial keypoints model
    df_predict = facial_keypoints_resnet_model.predict(test_imgs)
    
    # Make predictions using Emotion Detection model, and get the predicted class
    df_emotion = np.argmax(emotion_model.predict(test_imgs),axis =-1)
    
    # Reshape predictions of second model from (test_size,) to (test_size,1)
    df_emotion = np.expand_dims(df_emotion,axis = 1)
    
    # Convert predictions to a dataframe
    df_predict = pd.DataFrame(df_predict,columns = columns)
    
    df_predict['emotion'] = df_emotion
    
    return df_predict    

# Loading test split(of emotion data)

In [6]:
with open('trained_models/emotion_test_split.json','r') as f:
    to_decode = json.load(f)

test_data = json.loads(to_decode)

Before converting to numpy array, we check that the data has been read correctly.

In [11]:
len(test_data)

1229

In [12]:
len(test_data[0])

96

In [13]:
len(test_data[0][0])

96

Converting to numpy array

In [14]:
test_data = np.array(test_data)
test_data.shape

(1229, 96, 96, 1)

Testing our function

In [16]:
df_predict = predict(test_data)

In [17]:
df_predict

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,emotion
0,63.225338,39.555046,30.223234,39.981815,57.113300,40.158997,70.933685,39.934227,36.810547,40.570427,...,52.860905,62.059879,65.060501,33.955898,65.286751,47.806236,63.300556,47.314823,69.232315,3
1,63.858227,35.405937,29.519472,34.745228,57.431396,36.362804,71.780769,36.353031,36.513950,36.072594,...,56.594479,61.833500,75.982635,32.601440,76.591232,47.349449,73.508614,47.029903,83.464516,2
2,64.983780,36.059624,28.473663,35.799683,57.658897,36.934914,73.093918,36.765244,36.239868,37.025284,...,57.662708,63.431553,75.867180,31.864979,76.845558,47.708309,73.973305,47.553303,82.645836,2
3,66.266068,57.231506,29.225914,56.698650,60.839813,56.197315,73.957932,56.405087,36.376297,56.147694,...,42.383160,63.708355,28.463791,33.326408,28.859207,48.518372,31.559700,48.341076,24.951239,3
4,62.588993,36.424339,32.766506,36.414913,56.968884,37.341351,68.973686,37.263817,39.194931,37.270084,...,58.718033,60.969318,77.897827,35.731026,78.042366,48.430191,75.640511,48.110439,84.221992,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,65.335052,36.820282,29.904051,36.129154,59.144066,37.561111,72.583282,37.737061,36.736530,37.204342,...,57.568581,62.365894,76.098717,33.432579,76.203751,48.003929,72.486702,47.572701,83.815147,0
1225,60.907719,36.744831,34.395382,37.152073,56.075356,37.824970,67.708221,37.960762,40.064674,38.493389,...,58.298260,60.338192,75.510033,36.752808,75.586006,48.479561,73.686722,47.699680,80.789421,2
1226,49.301739,39.464100,43.757874,39.889622,49.705399,39.996403,51.482235,39.692917,46.079304,40.380741,...,53.623756,50.438831,66.285538,46.099663,66.114250,47.787155,64.179054,47.003986,71.314827,4
1227,66.283669,58.997826,30.279085,58.659847,60.486519,58.462868,73.628334,58.614807,37.130314,58.384785,...,41.898853,64.289200,24.783585,34.311485,25.512850,49.311924,29.098455,48.765415,20.242533,2


Now we are able to use to use both the models and make predictions successfully

# Deploying the models using Tensorflow 2.0 serving

Steps 

1) Serialize and save models to be ready for deplyment using a deploy function

2) Install necessary packages

3) Assign ports and call deploy function for both models separately

4) Verify successful deployment by observing logs

5) Make json requests to deployed models using python's requests module

5) Get predictions from response and try to visualize


Step 1)

In [19]:
def deploy(dir,model):
    
    MODEL_DIR = dir
    
    version = 1
    
    export_path = os.path.join(MODEL_DIR,str(version))
    # for verification
    print(f"Export path : {export_path}\n")
    
    # Remove similar versions of model if already present
    if os.path.isdir(export_path):
        print("Deleting already available model of same version")
        !rm -r {export_path}
        
    tf.saved_model.save(model,export_path)
    
    os.environ["MODEL_DIR"] = MODEL_DIR

Step 2)

We need to add tensorflow-model-server package